## Setup

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
import boto3
import os
import re
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as style
from typing import List, Union
from pathlib import Path
pd.set_option('display.max_columns', 500)

In [3]:
path_data = Path.home() / 'data/site'
path_images = path_data / 'images'

## Get metadata

In [5]:
df = pd.read_json(path_data / 'raw' / 'metadata.json')

### From S3

In [346]:
boto3.setup_default_session(profile_name="crayon-site")
s3 = boto3.resource("s3")
s3_client = boto3.client("s3")

## EDA

### Basic Statistics

Look at some basic statistics for the metadata of the images.

In [6]:
df.head(1).T

,20_IL_1902_DBM6597911224/Mapping/DBI2168114984_DBI-136-YY9x6QBf-DJI_0918.JPG
ExifTool:ExifToolVersion,11.88
ExifTool:Warning,[minor] Possibly incorrect maker notes offsets...
File:FileName,DBI2168114984_DBI-136-YY9x6QBf-DJI_0918.JPG
File:Directory,20_IL_1902_DBM6597911224/Mapping
File:FileSize,8562206
...,...
Composite:DateTimeCreated,None
Composite:DOF,None
File:Directory1,20_IL_1902_DBM6597911224
File:Directory2,Mapping


Look at the mean and std of the altitude columns based on the specific site.

In [24]:
df_alt = df.loc[:,list(df.columns[df.columns.str.contains('Altitude|Directory1')].values)]

In [27]:
df_alt.groupby(['File:Directory1']).agg(['mean', 'std'])

EXIF:GPSAltitudeRef            \
                                                             mean       std   
File:Directory1                                                               
                                                          0.00000  0.000000   
1086                                                      1.00000  0.000000   
20_AZ_1066_DBM3174640504                                  0.00000  0.000000   
20_AZ_1069_DBM3916895777                                  0.00000  0.000000   
20_CA_1327_DBM2236713281                                  1.00000  0.000000   
20_IL_1875_DBM4642991424                                  0.00000  0.000000   
20_IL_1902_DBM6597911224                                  0.00000  0.000000   
20_IN_1037_DBM5218738239                                  0.00000  0.000000   
20_IN_1488_DBM0343529591                                  0.00000  0.000000   
20_KY_1317_DBM8839393143                                  0.00000  0.000000   
20_KY_643_DBM2303396163                                   0.00000  0.000000   
20_OH_1825_DBM9809148162                                  0.00000  0.000000   
20_OR_1088_DBM1093526169                                  1.00000  0.000000   
20_TN_1362_DBM7908447035                                  0.00000  0.000000   
20_TX_1263_DBM8876429465                                  0.00000  0.000000   
20_TX_1374_DBM2789862228                                  0.00000  0.000000   
20_TX_362_DBM9850599032                                   0.00000  0.000000   
20_TX_600_DBM3157935510                                   0.00000  0.000000   
20_TX_790_DBM8195085916                                   0.00000  0.000000   
20_WI_1853_DBM1831755914                                  0.00000  0.000000   
Addison_3_DBM1828917620                                   0.00000  0.000000   
Fairfield_4_DBM9902142293                                 0.00000  0.000000   
Rush_Boise                                                0.00000  0.000000   
Walking_Capture_Sites20_CA_1326_DBM6766509589             0.89898  0.301405   
Walking_Capture_Sites20_IN_1353_DBM8662320789             0.00000  0.000000   
neshaminy_mall_2020_local                                 0.00000  0.000000   
rush_effingham_2020                                       0.00000  0.000000   
valley_plaza_2020_DBM5959821491                           0.00000  0.000000   
wmdc6024_grovecityoh                                      0.00000  0.000000   
wmdc7030_pottsvillepa                                     0.00000  0.000000   

                                              EXIF:GPSAltitude             \
                                                          mean        std   
File:Directory1                                                             
                                                    154.273562   2.284722   
1086                                                 26.772112   1.465633   
20_AZ_1066_DBM3174640504                            375.138075   0.693437   
20_AZ_1069_DBM3916895777                            351.044797   0.064208   
20_CA_1327_DBM2236713281                             18.612926   0.433817   
20_IL_1875_DBM4642991424                            198.787899   0.720024   
20_IL_1902_DBM6597911224                            142.563228   0.074684   
20_IN_1037_DBM5218738239                            260.940252   0.682972   
20_IN_1488_DBM0343529591                             87.016144  17.104732   
20_KY_1317_DBM8839393143                            347.817611   2.051157   
20_KY_643_DBM2303396163                             337.951638   0.161997   
20_OH_1825_DBM9809148162                            272.551638   2.093983   
20_OR_1088_DBM1093526169                              3.139086   0.046863   
20_TN_1362_DBM7908447035                            189.689626   0.497825   
20_TX_1263_DBM8876429465                            139.016588   0.111154   
20_TX_1374_DBM2789862228                            